In [1]:
# import some function we can use later
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle

In [2]:
# import pandas to process the cvs data
import pandas as pd

In [9]:
! ls /home/htan/proj/TensorFlow/data/visibility

ASOS_alone.pickle	 KORD_RUC_RAP_Hourly_20051031-20150301.csv
ASOS_time_serial.pickle  rough_visibility.pickle
CC_Under_sSample.pickle  visibilityDataNoLagsNewPreds.csv


# Read ASOS data from csv file

In [11]:
filename_ASOS = '/home/htan/proj/TensorFlow/data/visibility/' +  'visibilityDataNoLagsNewPreds.csv'
ASOS_content = pd.read_csv(filename_ASOS)
print(ASOS_content)



                    Time  KRFD_FTD_0  KRFD_U_0  KRFD_V_0  KRFD_VIS_0  \
0       04/01/2005 00:00    0.000000  0.000000  0.000000        10.0   
1       04/01/2005 00:15    0.006250  0.000000  0.000000        10.0   
2       04/01/2005 00:30    0.012500  0.000000  0.000000        10.0   
3       04/01/2005 00:45    0.018750  0.000000  0.000000        10.0   
4       04/01/2005 01:00    0.041667  0.000000  0.000000        10.0   
5       04/01/2005 01:15    0.047917  0.000000  0.000000        10.0   
6       04/01/2005 01:30    0.054167  0.000000  0.000000        10.0   
7       04/01/2005 01:45    0.060417  0.000000  0.000000        10.0   
8       04/01/2005 02:00    0.083333  0.000000  0.000000        10.0   
9       04/01/2005 02:15    0.089583  0.000000  0.000000        10.0   
10      04/01/2005 02:30    0.095833  0.000000  0.000000        10.0   
11      04/01/2005 02:45    0.102083 -1.026060 -2.819078        10.0   
12      04/01/2005 03:00    0.125000  0.000000  0.000000        

# Read Numerical Weather Prediction data from csv file

In [12]:
filename_NWP = '/home/htan/proj/TensorFlow/data/visibility/' + 'KORD_RUC_RAP_Hourly_20051031-20150301.csv'
NWP_content = pd.read_csv(filename_NWP)
print(NWP_content)

       YEAR  MONTH  DAY  TIME  HOUR_003  HOUR_004  HOUR_005  RHPBL_003  \
0      2005      4    1     0       300       400       500        NaN   
1      2005      4    1   100       400       500       600        NaN   
2      2005      4    1   200       500       600       700        NaN   
3      2005      4    1   300       600       700       800        NaN   
4      2005      4    1   400       700       800       900        NaN   
5      2005      4    1   500       800       900      1000        NaN   
6      2005      4    1   600       900      1000      1100        NaN   
7      2005      4    1   700      1000      1100      1200        NaN   
8      2005      4    1   800      1100      1200      1300        NaN   
9      2005      4    1   900      1200      1300      1400        NaN   
10     2005      4    1  1000      1300      1400      1500        NaN   
11     2005      4    1  1100      1400      1500      1600        NaN   
12     2005      4    1  1200      150

In [14]:
ASOS_data = ASOS_content.values
NWP_data = NWP_content.values
ASOS_data.shape, NWP_data.shape

((347616, 72), (86928, 19))

In [38]:
# match to data set by looking at the time series data
NWP_data_adjust = np.repeat(NWP_data[:86904, 7:], 4, axis = 0)
NWP_data_adjust.shape

(347616, 12)

In [54]:
NWP_data_3hours = NWP_data_adjust[:, 0:-1:3]

In [55]:
NWP_data_3hours[12650*4:12650*4+1, :]

array([[ 92.197,  17.65 ,  15.9  ,   0.   ]])

In [39]:
NWP_data_adjust[12650*4:12650*4+1, :]

array([[ 92.197,     nan,     nan,  17.65 ,     nan,     nan,  15.9  ,
            nan,     nan,   0.   ,     nan,     nan]])

In [63]:
time_adjust = ASOS_data[:, 0:1]
ASOS_data_adjust = ASOS_data[:, 1:-1]
print(time_adjust.shape, ASOS_data_adjust.shape)

(347616, 1) (347616, 70)


In [76]:
# get the label
label = ASOS_content['KORD_V_0']
vis_label = label.values
vis_label_1hour_p = vis_label[4:]
print (vis_label_1hour_p.shape)
print(vis_label_1hour_p[:20])
vis_label_1hour_p_f = np.append(vis_label_1hour_p, [np.nan]*4)
print(vis_label_1hour_p_f.shape)
print(vis_label_1hour_p_f)


(347612,)
[  9.  10.  10.   9.   8.   9.   8.   7.   7.   6.   6.   7.   7.   7.   9.
   8.   7.   7.   7.  10.]
(347616,)
[  9.  10.  10. ...,  nan  nan  nan]


In [80]:
vis_label_1hour_p_f.reshape(-1, 1).shape

(347616, 1)

In [85]:
total_data = np.hstack((ASOS_data_adjust, NWP_data_adjust, vis_label_1hour_p_f.reshape(-1, 1)))
total_data.shape

(347616, 83)

In [86]:
#change the type from object to float for calculation 
temp_tt = total_data.astype(float)
#find the nan value
total_b = np.isnan(temp_tt).any(axis=1)

In [87]:
total_data_final = temp_tt[~total_b]
time_adjust_final = time_adjust[~total_b]
total_data_final.shape, time_adjust_final.shape

((145822, 83), (145822, 1))

In [88]:
time_adjust_final[:10]

array([['04/13/2006 21:00'],
       ['04/13/2006 21:15'],
       ['04/13/2006 21:30'],
       ['04/13/2006 21:45'],
       ['10/08/2006 15:00'],
       ['10/08/2006 15:15'],
       ['10/08/2006 15:30'],
       ['10/08/2006 15:45'],
       ['10/08/2006 16:00'],
       ['10/08/2006 16:15']], dtype=object)

In [89]:
total_data_final[:10, -1]

array([  7.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.])

In [90]:
ASOS_data.shape, NWP_data_adjust.shape, 

((347616, 72), (347616, 12))

In [91]:
length = total_data_final.shape[0]
length_list = range(0, length)
train_len = length * 7 / 10
validate_len = length / 10 
test_len = length - train_len - validate_len
print(train_len, validate_len, test_len)

102075 14582 29165


In [95]:
 total_data_final[0, 69]

10.0

In [92]:
train_list = length_list[0:train_len]
validate_list = length_list[train_len:train_len + validate_len]
test_list = length_list[train_len + validate_len:]
sum_v_t = length_list[train_len:]
sum_t_v = length_list[:train_len + validate_len]
print(len(train_list), len(validate_list), len(test_list), len(sum_v_t), len(sum_t_v))
print(train_list[0], validate_list[0], test_list[0])

102075 14582 29165 43747 116657
0 102075 116657


In [99]:
train_dataset = {
    'time' : time_adjust_final[train_list],
    'data_ASOS' : total_data_final[train_list, :70],
    'data_NWP': total_data_final[train_list, 70:-1],
    'label' : total_data_final[train_list, -1],
}
validate_dataset= {
    'time' : time_adjust_final[validate_list],
    'data_ASOS' : total_data_final[validate_list, :70],
    'data_NWP' : total_data_final[validate_list, 70:-1],
    'label' : total_data_final[validate_list, -1],
}
test_dataset = {
    'time' : time_adjust_final[test_list],
    'data_ASOS' : total_data_final[test_list, :70],
    'data_NWP' : total_data_final[test_list, 70:-1],
    'label' : total_data_final[test_list, -1],
}
v_t_dataset = {
    'time' : time_adjust_final[sum_v_t],
    'data_ASOS' : total_data_final[sum_v_t, :70],
    'data_NWP' : total_data_final[sum_v_t, 70:-1],
    'label' : total_data_final[sum_v_t, -1],
}
t_v_dataset = {
    'time' : time_adjust_final[sum_t_v],
    'data_ASOS' : total_data_final[sum_t_v, :70],
    'data_NWP' : total_data_final[sum_t_v, 70:-1],
    'label' : total_data_final[sum_t_v, -1],
}

In [101]:
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS+NWP.pickle'

with open(pickle_file, 'wb') as f:
    save = {
        'train_dataset': train_dataset,
        'validate_dataset': validate_dataset,
        'test_dataset': test_dataset,
        'v_t_dataset': v_t_dataset,
        't_v_dataset':t_v_dataset,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)


Create time serials data for RNN

In [6]:
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS+NWP.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    test_dataset = save['test_dataset']
    train_old = save['t_v_dataset']
    del save

test_time = test_dataset['time']
test_data_1 = test_dataset['data_ASOS']
test_data_2 = test_dataset['data_NWP']
test_label = test_dataset['label']

train_time = train_old['time']
train_data_1 = train_old['data_ASOS']
train_data_2 = train_old['data_NWP']
train_label = train_old['label']

In [9]:
train_time.shape, test_time.shape

((116657, 1), (29165, 1))

In [11]:
data = np.hstack((np.vstack((train_data_1, test_data_1)), np.vstack((train_data_2, test_data_2))))


In [22]:
time = np.concatenate((train_time, test_time)).reshape(-1)
label = np.hstack((train_label, test_label))
data.shape, time.shape, label.shape

((145822, 82), (145822,), (145822,))

In [24]:
time[:100]

array(['04/13/2006 21:00', '04/13/2006 21:15', '04/13/2006 21:30',
       '04/13/2006 21:45', '10/08/2006 15:00', '10/08/2006 15:15',
       '10/08/2006 15:30', '10/08/2006 15:45', '10/08/2006 16:00',
       '10/08/2006 16:15', '10/08/2006 16:30', '10/08/2006 17:00',
       '10/08/2006 17:15', '10/08/2006 17:30', '10/08/2006 17:45',
       '10/08/2006 18:00', '10/08/2006 18:15', '10/08/2006 18:30',
       '10/08/2006 18:45', '10/08/2006 19:00', '10/08/2006 19:15',
       '10/08/2006 19:30', '10/08/2006 19:45', '10/08/2006 20:00',
       '10/08/2006 20:15', '10/08/2006 20:30', '10/08/2006 20:45',
       '10/08/2006 21:00', '10/08/2006 21:15', '10/08/2006 21:30',
       '10/08/2006 21:45', '10/08/2006 22:00', '10/08/2006 22:45',
       '10/08/2006 23:00', '10/08/2006 23:30', '10/08/2006 23:45',
       '10/09/2006 00:00', '10/09/2006 00:15', '10/09/2006 00:30',
       '10/09/2006 00:45', '10/09/2006 01:00', '10/09/2006 01:15',
       '10/09/2006 01:30', '10/09/2006 01:45', '10/09/2006 02:

In [25]:
import time as t
for i in range(1, 100):
    t_diff = t.mktime(t.strptime(time[i], "%m/%d/%Y %H:%M")) - t.mktime(t.strptime(time[i-1], "%m/%d/%Y %H:%M"))
    print(t_diff)

900.0
900.0
900.0
15354900.0
900.0
900.0
900.0
900.0
900.0
900.0
1800.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
2700.0
900.0
1800.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
1800.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
1800.0
900.0
900.0
1800.0
900.0
900.0
900.0
900.0
900.0
1800.0
12600.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0
900.0


In [26]:
n_steps = 8
n_samples = 25000
print(n_samples)

25000


In [29]:
class rnn_data_generator:
    
    def __init__(self, time, data, label):
        self.time = time
        self.data = data
        self.label = label
        self.s_data = data[:n_steps].reshape(1, n_steps, data.shape[1])
        self.s_label = label[:n_steps].reshape(1, n_steps, 1)
        self.rnn_data = self.s_data
        self.rnn_label = self.s_label
        self.rnn_datas = []
        self.rnn_labels = []
        
    def generate(self, number_batches = 10):
        n_samples = self.time.shape[0] / 10
        for n in range(number_batches):
            false_count = 0
            for i in range(n_samples - 10):
                offset = i + n_samples*n
                temp_data = data[offset:offset+n_steps].reshape(1, n_steps, data.shape[1])
                temp_label = label[offset:offset+n_steps].reshape(1, n_steps, 1)
                no_time_gap = True
                for j in range(n_steps):
                    t_diff = t.mktime(t.strptime(time[offset+j], "%m/%d/%Y %H:%M")) - t.mktime(t.strptime(time[offset+j-1], "%m/%d/%Y %H:%M"))
                    if t_diff >= 950.0:
                        false_count += 1
                        no_time_gap = False
                        break
                if no_time_gap:
                    self.rnn_data = np.vstack((self.rnn_data, temp_data))
                    self.rnn_label = np.vstack((self.rnn_label, temp_label))
                    self.s_data = temp_data
                    self.s_label = temp_label
            
            self.rnn_datas.append(self.rnn_data)
            self.rnn_labels.append(self.rnn_label)
            self.rnn_data = self.s_data
            self.rnn_label = self.s_label

In [30]:
g = rnn_data_generator(time, data, label)

In [31]:
g.generate(10)

In [32]:
[s.shape for s in g.rnn_datas]

[(8548, 8, 82),
 (7806, 8, 82),
 (9149, 8, 82),
 (9312, 8, 82),
 (8862, 8, 82),
 (9640, 8, 82),
 (8567, 8, 82),
 (7884, 8, 82),
 (9780, 8, 82),
 (8772, 8, 82)]

In [33]:
[s.shape for s in g.rnn_labels]

[(8548, 8, 1),
 (7806, 8, 1),
 (9149, 8, 1),
 (9312, 8, 1),
 (8862, 8, 1),
 (9640, 8, 1),
 (8567, 8, 1),
 (7884, 8, 1),
 (9780, 8, 1),
 (8772, 8, 1)]

In [34]:
rnn_data_t = np.vstack(g.rnn_datas)
rnn_label_t = np.vstack(g.rnn_labels)

In [35]:
rnn_data_t.shape, rnn_label_t.shape

((88320, 8, 82), (88320, 8, 1))

In [36]:
length = rnn_data_t.shape[0]
length_list = range(0, length)
train_len = length * 7 / 10
validate_len = length / 10 
test_len = length - train_len - validate_len
print(train_len, validate_len, test_len)

61824 8832 17664


In [37]:
train_list = length_list[0:train_len]
validate_list = length_list[train_len:train_len + validate_len]
test_list = length_list[train_len + validate_len:]
sum_v_t = length_list[train_len:]
sum_t_v = length_list[:train_len + validate_len]
print(len(train_list), len(validate_list), len(test_list), len(sum_v_t), len(sum_t_v))
print(train_list[0], validate_list[0], test_list[0])

61824 8832 17664 26496 70656
0 61824 70656


In [38]:
train_dataset = {
    'data' : rnn_data_t[train_list, :, :],
    'label' : rnn_label_t[train_list, :, :],
}
validate_dataset= {
    'data' : rnn_data_t[validate_list, :, :],
    'label' : rnn_label_t[validate_list, :, :],
}
test_dataset = {
    'data' : rnn_data_t[test_list, :, :],
    'label' : rnn_label_t[test_list, :, :],
}
v_t_dataset = {
    'data' : rnn_data_t[sum_v_t, :, :],
    'label' : rnn_label_t[sum_v_t, :, :],
}
t_v_dataset = {
    'data' : rnn_data_t[sum_t_v, :, :],
    'label' : rnn_label_t[sum_t_v, :, :],
}

In [39]:
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS+NWP_time_serial.pickle'

with open(pickle_file, 'wb') as f:
    save = {
        'train_dataset': train_dataset,
        'validate_dataset': validate_dataset,
        'test_dataset': test_dataset,
        'v_t_dataset': v_t_dataset,
        't_v_dataset':t_v_dataset,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
